# Collecting arxiv abstract information

In this notebook I collect arxiv information from the abstract pages. No links are followed, thus no crawling is needed. 

In [1]:
from  scrapy import Selector
import requests
import time as time
import numpy as np
import pandas as pd


In [77]:
#function to go trough single one 
def get_single_abstract(url,silent=False):
    check=requests.get(url)
    #status exist if 200 or below 400
    if check.status_code==200:
        #result list which is collects
        results=[]
        id_is=url.split('/')[-1]
        #website id
        results.append(id_is)
        #abstract number excluding version
        if id_is.find('v')>-1:
            t=id_is.split('v')
            results.append(t[0])
            #whetehr final 
            results.append("False")
        else:
            results.append(id_is)
            results.append("True")            
        if silent==False:
            print(f"{url} can be accessed")
        html=check.content
        sel=Selector(text=html)
        #abstract selection
        sel2=sel.xpath('//blockquote//text()')
        #extract content
        p2=sel2.extract()
        #get abstract content replace new line with space 
        abstract=p2[2].replace('\n',' ').strip()
        if silent==False:
            print(abstract)
        #get title
        sel3=sel.xpath('//*[@class="title mathjax"]//text()')
        p3=sel3.extract()
        #get title content replace new line with space 
        title=p3[1].replace('\n',' ').strip()
        results.append(title)
        results.append(abstract)        
        if silent==False:
            print(title)
        #select authors
        sel4=sel.xpath('//*[@class="authors"]//text()')
        p4=sel4.extract()
        #get elements which are not authors or comma or empty elements
        authors=[]
        for x in p4:
            if x!="Authors:" and x!=', ':
                y=x.strip()
                if y!='':
                    authors.append(x.strip())
        results.append(len(authors))   
        results.append(authors)          
        if silent==False:
            print(len(authors),authors)
        #get comments
        sel5=sel.xpath('//*[@class="tablecell comments mathjax"]//text()')
        #extract them 
        if len(sel5)==1:
            comments=sel5.extract()[0].strip()
        else:
            comments=None
        if silent==False:
            #print them 
            print(comments)
        results.append(comments)              
        #subject primary
        sel6=sel.xpath('//*[@class="primary-subject"]//text()')
        prim_subject=sel6.extract()[0]
        results.append(prim_subject)          
        if silent==False:
            #print them 
            print(prim_subject)
        #all subjects (doubles previous  to be figured with no other how to combine them )
        sel7=sel.xpath('//*[@class="tablecell subjects"]//text()')
        p7=sel7.extract()
        #exclude first is new line second is primary gotten above 
        if len(p7)==3:
            subjects=p7[2].replace(";","").replace("'","")
            #split them 
            s_subjects_prel=subjects.split(")")
            #adds back ) and excludes last one 
            n_subjects=len(s_subjects_prel)
            s_subjects=[]
            for i in range(len(s_subjects_prel)-1):
                s_subjects.append(s_subjects_prel[i]+")")
            if silent==False:    
                print(s_subjects)
        else:
            s_subjects=None
            n_subjects=0
        results.append(n_subjects)
        results.append(s_subjects)           
        #should be split by ; when not empty
        if silent==False:
            print(n_subjects,s_subjects)   
        #to add submission time? and size?    
        #getting version string if not export version 
        if id_is.find('export')>-1:
            sel8=sel.xpath('//*[@class="submission-history"]//b/text()')
        else:
            sel8=sel.xpath('//*[@class="submission-history"]//strong/text()')
        sel8b=sel.xpath('//*[@class="submission-history"]/text()')
        #version number
        vlast=sel8.extract()[0].strip('[').strip(']').strip('v')
        if silent==False:
            print(vlast)
        #version properties
        vprop=sel8b.extract()
        vsel=vprop[-2].strip()
        if silent==False:
            print(vsel)
        #append vesrion number number
        results.append(int(vlast))  
        #append version properties
        results.append(vsel)
        return results, 0
    else:
        print(f"{url} cannot be accessed, status is {check.status_code}")
        return None, 1

In [26]:
#some special (big author list) retracted first
urls=['https://export.arxiv.org/abs/1703.00052','https://export.arxiv.org/abs/2010.10965','https://export.arxiv.org/abs/2010.10966','https://export.arxiv.org/abs/2010.10967']
#make them automatic , 404 when obviously not extsing
#403 access denied when to many thus now xport used 

In [27]:
#the long author list cause problems not clear whether needed to solve 
start_time=time.time()
for url in urls:
    # list of strings
    res,check=get_single_abstract(url)
stop_time=time.time()
print(f"{len(urls)} sites needed {np.round(stop_time-start_time,3)} seconds")
#not vsisible authors are visisble here but also not numbers 

https://arxiv.org/abs/1703.00052 can be accessed
We describe the Sloan Digital Sky Survey IV (SDSS-IV), a project encompassing three major spectroscopic programs. The Apache Point Observatory Galactic Evolution Experiment 2 (APOGEE-2) is observing hundreds of thousands of Milky Way stars at high resolution and high signal-to-noise ratio in the near-infrared. The Mapping Nearby Galaxies at Apache Point Observatory (MaNGA) survey is obtaining spatially-resolved spectroscopy for thousands of nearby galaxies (median redshift of z = 0.03). The extended Baryon Oscillation Spectroscopic Survey (eBOSS) is mapping the galaxy, quasar, and neutral gas distributions between redshifts z = 0.6 and 3.5 to constrain cosmology using baryon acoustic oscillations, redshift space distortions, and the shape of the power spectrum. Within eBOSS, we are conducting two major subprograms: the SPectroscopic IDentification of eROSITA Sources (SPIDERS), investigating X-ray AGN and galaxies in X-ray clusters, and t

https://arxiv.org/abs/1804.09365 can be accessed
We present the second Gaia data release, Gaia DR2, consisting of astrometry, photometry, radial velocities, and information on astrophysical parameters and variability, for sources brighter than magnitude 21. In addition epoch astrometry and photometry are provided for a modest sample of minor planets in the solar system.
Gaia Data Release 2. Summary of the contents and survey properties
14 ['Gaia Collaboration', ':', 'A.G.A. Brown', '(1),', 'A. Vallenari', '(2),', 'T. Prusti', '(3),', 'J.H.J. de Bruijne', '(3),', 'C. Babusiaux', '(4,5),', 'C.A.L. Bailer-Jones', '(6) ((1) Leiden Observatory, Leiden University, (2) INAF - Osservatorio astronomico di Padova, (3) ESA/ESTEC, (4) GEPI, Observatoire de Paris, (5) Univ. Grenoble Alpes, (6) Max Planck Institute for Astronomy)']
Accepted for A&A Special Issue on Gaia DR2, 21 pages, 9 figures, high resolution Milky Way sky maps available from ESA Gaia web pages, 447 co-authors
Astrophysics of Gala

https://arxiv.org/abs/2010.00001 can be accessed
AE Aqr objects are a class of cataclysmic variable stars in which the rapidly rotating magnetosphere of the white dwarf (WD) primary centrifugally expels most infalling gas before it can accrete onto the WD. The expulsion of the accretion flow via this "magnetic propeller" extracts angular momentum from the WD and produces large-amplitude, aperiodic flares in optical photometry. The eponymous AE Aqr is the only confirmed member of this class of object, but recently, Thorstensen (2020) discovered a candidate AE Aqr system: LAMOST J024048.51+195226.9. Using survey photometry, we measure a refined orbital period for this system and identify a shallow, previously unrecognized eclipse during which the system's frequent AE Aqr-like flaring episodes cease. A dedicated follow-up study is still necessary to test the proposed AE Aqr classification for LAMOST J024048.51+195226.9, but should it be confirmed, the eclipse of its flare-production regio

In [28]:
print(res)

['2010.10967', '2010.10967', 'Safe Handover in Mixed-Initiative Control for Cyber-Physical Systems', 'For mixed-initiative control between cyber-physical systems (CPS) and its users, it is still an open question how machines can safely hand over control to humans. In this work, we propose a concept to provide technological support that uses formal methods from AI -- description logic (DL) and automated planning -- to predict more reliably when a hand-over is necessary, and to increase the advance notice for handovers by planning ahead of runtime. We combine this with methods from human-computer interaction (HCI) and natural language generation (NLG) to develop solutions for safe and smooth handovers and provide an example autonomous driving scenario. A study design is proposed with the assessment of qualitative feedback, cognitive load and trust in automation.', 10, ['Frederik Wiehr', 'Anke Hirsch', 'Florian Daiber', 'Antonio Kruger', 'Alisa Kovtunova', 'Stefan Borgwardt', 'Ernie Chang

Looks now good enough at least for standard author lists. Now I build a function which gets all these for all urls starting with one and getting also the previous versions and the following numbers, until some break condition is reached. 

In [81]:
# input starturl, any no v abstratc works, it does not carry over the next month, max_number of primary, number of hours run
#whether get_single_abstract is silent

def get_many(starturl,max_number=20,max_h=1,silent=True):
    start_time=time.time()
    counter=0
    # list of strings
    print(starturl)
    res,check=get_single_abstract(starturl,silent=silent)
    if check==0:
        df2=pd.DataFrame(res,index=['Id','Abstract-number','Final','Title','Abstract','Author_number','Authors','Comments','Prim_subject','n_sec_subjects','sec_subjects','version_number','version_properties']).T
        counter+=1
    check_time=time.time()
    #as long as time o.k. and iteration o.k. and scrap worked
    while (check_time-start_time)/3600<max_h and counter<max_number and check==0:
        print(counter)
        new_url=starturl[:-5]+str(int(starturl[-5:])+counter).zfill(5)
        print(new_url)
        res,check=get_single_abstract(new_url,silent=silent)
        check_time=time.time()
        if check==0:
            df=pd.DataFrame(res,index=['Id','Abstract-number','Final','Title','Abstract','Author_number','Authors','Comments','Prim_subject','n_sec_subjects','sec_subjects','version_number','version_properties']).T
            df2=pd.concat([df2,df],axis=0,ignore_index=True)
            counter+=1          
            vcounter=0
            #looping over versions
            print(f" version number is {int(df.loc[0,['version_number']])}")
            if int(df.loc[0,['version_number']])>1:
                while int(df.loc[0,['version_number']])>1:            
                    urlb=new_url+'v'+str(int(df.loc[0,['version_number']]-1))
                    print(urlb)
                    res,check=get_single_abstract(urlb,silent=silent)
                    df=pd.DataFrame(res,index=['Id','Abstract-number','Final','Title','Abstract','Author_number','Authors','Comments','Prim_subject','n_sec_subjects','sec_subjects','version_number','version_properties']).T        
                    print(f" version number is {int(df.loc[0,['version_number']])}")
                    df2=pd.concat([df2,df],axis=0,ignore_index=True)       
    return df2

In [83]:
df3=get_many('https://export.arxiv.org/abs/2001.05775',max_number=10000,max_h=3,silent=True)
df3.to_csv('abstract_2001_part3.csv', sep=',')


https://export.arxiv.org/abs/2001.00658
1
https://export.arxiv.org/abs/2001.00659
 version number is 2
https://export.arxiv.org/abs/2001.00659v1
 version number is 1
2
https://export.arxiv.org/abs/2001.00660
 version number is 1
3
https://export.arxiv.org/abs/2001.00661
 version number is 1
4
https://export.arxiv.org/abs/2001.00662
 version number is 2
https://export.arxiv.org/abs/2001.00662v1
 version number is 1
5
https://export.arxiv.org/abs/2001.00663
 version number is 1
6
https://export.arxiv.org/abs/2001.00664
 version number is 5
https://export.arxiv.org/abs/2001.00664v4
 version number is 4
https://export.arxiv.org/abs/2001.00664v3
 version number is 3
https://export.arxiv.org/abs/2001.00664v2
 version number is 2
https://export.arxiv.org/abs/2001.00664v1
 version number is 1
7
https://export.arxiv.org/abs/2001.00665
 version number is 2
https://export.arxiv.org/abs/2001.00665v1
 version number is 1
8
https://export.arxiv.org/abs/2001.00666
 version number is 1
9
https://expor

 version number is 1
76
https://export.arxiv.org/abs/2001.00734
 version number is 1
77
https://export.arxiv.org/abs/2001.00735
 version number is 2
https://export.arxiv.org/abs/2001.00735v1
 version number is 1
78
https://export.arxiv.org/abs/2001.00736
 version number is 1
79
https://export.arxiv.org/abs/2001.00737
 version number is 1
80
https://export.arxiv.org/abs/2001.00738
 version number is 1
81
https://export.arxiv.org/abs/2001.00739
 version number is 1
82
https://export.arxiv.org/abs/2001.00740
 version number is 1
83
https://export.arxiv.org/abs/2001.00741
 version number is 1
84
https://export.arxiv.org/abs/2001.00742
 version number is 1
85
https://export.arxiv.org/abs/2001.00743
 version number is 3
https://export.arxiv.org/abs/2001.00743v2
 version number is 2
https://export.arxiv.org/abs/2001.00743v1
 version number is 1
86
https://export.arxiv.org/abs/2001.00744
 version number is 3
https://export.arxiv.org/abs/2001.00744v2
 version number is 2
https://export.arxiv.or

 version number is 1
155
https://export.arxiv.org/abs/2001.00813
 version number is 1
156
https://export.arxiv.org/abs/2001.00814
 version number is 2
https://export.arxiv.org/abs/2001.00814v1
 version number is 1
157
https://export.arxiv.org/abs/2001.00815
 version number is 3
https://export.arxiv.org/abs/2001.00815v2
 version number is 2
https://export.arxiv.org/abs/2001.00815v1
 version number is 1
158
https://export.arxiv.org/abs/2001.00816
 version number is 2
https://export.arxiv.org/abs/2001.00816v1
 version number is 1
159
https://export.arxiv.org/abs/2001.00817
 version number is 1
160
https://export.arxiv.org/abs/2001.00818
 version number is 1
161
https://export.arxiv.org/abs/2001.00819
 version number is 3
https://export.arxiv.org/abs/2001.00819v2
 version number is 2
https://export.arxiv.org/abs/2001.00819v1
 version number is 1
162
https://export.arxiv.org/abs/2001.00820
 version number is 1
163
https://export.arxiv.org/abs/2001.00821
 version number is 1
164
https://expo

 version number is 2
https://export.arxiv.org/abs/2001.00886v1
 version number is 1
229
https://export.arxiv.org/abs/2001.00887
 version number is 2
https://export.arxiv.org/abs/2001.00887v1
 version number is 1
230
https://export.arxiv.org/abs/2001.00888
 version number is 4
https://export.arxiv.org/abs/2001.00888v3
 version number is 3
https://export.arxiv.org/abs/2001.00888v2
 version number is 2
https://export.arxiv.org/abs/2001.00888v1
 version number is 1
231
https://export.arxiv.org/abs/2001.00889
 version number is 2
https://export.arxiv.org/abs/2001.00889v1
 version number is 1
232
https://export.arxiv.org/abs/2001.00890
 version number is 2
https://export.arxiv.org/abs/2001.00890v1
 version number is 1
233
https://export.arxiv.org/abs/2001.00891
 version number is 1
234
https://export.arxiv.org/abs/2001.00892
 version number is 1
235
https://export.arxiv.org/abs/2001.00893
 version number is 1
236
https://export.arxiv.org/abs/2001.00894
 version number is 1
237
https://export

 version number is 1
303
https://export.arxiv.org/abs/2001.00961
 version number is 1
304
https://export.arxiv.org/abs/2001.00962
 version number is 1
305
https://export.arxiv.org/abs/2001.00963
 version number is 1
306
https://export.arxiv.org/abs/2001.00964
 version number is 1
307
https://export.arxiv.org/abs/2001.00965
 version number is 1
308
https://export.arxiv.org/abs/2001.00966
 version number is 1
309
https://export.arxiv.org/abs/2001.00967
 version number is 1
310
https://export.arxiv.org/abs/2001.00968
 version number is 1
311
https://export.arxiv.org/abs/2001.00969
 version number is 3
https://export.arxiv.org/abs/2001.00969v2
 version number is 2
https://export.arxiv.org/abs/2001.00969v1
 version number is 1
312
https://export.arxiv.org/abs/2001.00970
 version number is 1
313
https://export.arxiv.org/abs/2001.00971
 version number is 1
314
https://export.arxiv.org/abs/2001.00972
 version number is 3
https://export.arxiv.org/abs/2001.00972v2
 version number is 2
https://ex

...

 version number is 1
4767
https://export.arxiv.org/abs/2001.05425
 version number is 1
4768
https://export.arxiv.org/abs/2001.05426
 version number is 1
4769
https://export.arxiv.org/abs/2001.05427
 version number is 3
https://export.arxiv.org/abs/2001.05427v2
 version number is 2
https://export.arxiv.org/abs/2001.05427v1
 version number is 1
4770
https://export.arxiv.org/abs/2001.05428
 version number is 2
https://export.arxiv.org/abs/2001.05428v1
 version number is 1
4771
https://export.arxiv.org/abs/2001.05429
 version number is 2
https://export.arxiv.org/abs/2001.05429v1
 version number is 1
4772
https://export.arxiv.org/abs/2001.05430
 version number is 1
4773
https://export.arxiv.org/abs/2001.05431
 version number is 1
4774
https://export.arxiv.org/abs/2001.05432
 version number is 3
https://export.arxiv.org/abs/2001.05432v2
 version number is 2
https://export.arxiv.org/abs/2001.05432v1
 version number is 1
4775
https://export.arxiv.org/abs/2001.05433
 version number is 1
4776
ht

 version number is 1
4845
https://export.arxiv.org/abs/2001.05503
 version number is 2
https://export.arxiv.org/abs/2001.05503v1
 version number is 1
4846
https://export.arxiv.org/abs/2001.05504
 version number is 1
4847
https://export.arxiv.org/abs/2001.05505
 version number is 2
https://export.arxiv.org/abs/2001.05505v1
 version number is 1
4848
https://export.arxiv.org/abs/2001.05506
 version number is 3
https://export.arxiv.org/abs/2001.05506v2
 version number is 2
https://export.arxiv.org/abs/2001.05506v1
 version number is 1
4849
https://export.arxiv.org/abs/2001.05507
 version number is 1
4850
https://export.arxiv.org/abs/2001.05508
 version number is 2
https://export.arxiv.org/abs/2001.05508v1
 version number is 1
4851
https://export.arxiv.org/abs/2001.05509
 version number is 1
4852
https://export.arxiv.org/abs/2001.05510
 version number is 2
https://export.arxiv.org/abs/2001.05510v1
 version number is 1
4853
https://export.arxiv.org/abs/2001.05511
 version number is 2
https:/

 version number is 5
https://export.arxiv.org/abs/2001.05574v4
 version number is 4
https://export.arxiv.org/abs/2001.05574v3
 version number is 3
https://export.arxiv.org/abs/2001.05574v2
 version number is 2
https://export.arxiv.org/abs/2001.05574v1
 version number is 1
4917
https://export.arxiv.org/abs/2001.05575
 version number is 1
4918
https://export.arxiv.org/abs/2001.05576
 version number is 1
4919
https://export.arxiv.org/abs/2001.05577
 version number is 2
https://export.arxiv.org/abs/2001.05577v1
 version number is 1
4920
https://export.arxiv.org/abs/2001.05578
 version number is 1
4921
https://export.arxiv.org/abs/2001.05579
 version number is 1
4922
https://export.arxiv.org/abs/2001.05580
 version number is 1
4923
https://export.arxiv.org/abs/2001.05581
 version number is 1
4924
https://export.arxiv.org/abs/2001.05582
 version number is 1
4925
https://export.arxiv.org/abs/2001.05583
 version number is 1
4926
https://export.arxiv.org/abs/2001.05584
 version number is 2
http

 version number is 1
4983
https://export.arxiv.org/abs/2001.05641
 version number is 1
4984
https://export.arxiv.org/abs/2001.05642
 version number is 1
4985
https://export.arxiv.org/abs/2001.05643
 version number is 10
https://export.arxiv.org/abs/2001.05643v9
 version number is 9
https://export.arxiv.org/abs/2001.05643v8
 version number is 8
https://export.arxiv.org/abs/2001.05643v7
 version number is 7
https://export.arxiv.org/abs/2001.05643v6
 version number is 6
https://export.arxiv.org/abs/2001.05643v5
 version number is 5
https://export.arxiv.org/abs/2001.05643v4
 version number is 4
https://export.arxiv.org/abs/2001.05643v3
 version number is 3
https://export.arxiv.org/abs/2001.05643v2
 version number is 2
https://export.arxiv.org/abs/2001.05643v1
 version number is 1
4986
https://export.arxiv.org/abs/2001.05644
 version number is 5
https://export.arxiv.org/abs/2001.05644v4
 version number is 4
https://export.arxiv.org/abs/2001.05644v3
 version number is 3
https://export.arxiv.

 version number is 3
https://export.arxiv.org/abs/2001.05711v2
 version number is 2
https://export.arxiv.org/abs/2001.05711v1
 version number is 1
5054
https://export.arxiv.org/abs/2001.05712
 version number is 1
5055
https://export.arxiv.org/abs/2001.05713
 version number is 2
https://export.arxiv.org/abs/2001.05713v1
 version number is 1
5056
https://export.arxiv.org/abs/2001.05714
 version number is 1
5057
https://export.arxiv.org/abs/2001.05715
 version number is 1
5058
https://export.arxiv.org/abs/2001.05716
 version number is 1
5059
https://export.arxiv.org/abs/2001.05717
 version number is 1
5060
https://export.arxiv.org/abs/2001.05718
 version number is 2
https://export.arxiv.org/abs/2001.05718v1
 version number is 1
5061
https://export.arxiv.org/abs/2001.05719
 version number is 2
https://export.arxiv.org/abs/2001.05719v1
 version number is 1
5062
https://export.arxiv.org/abs/2001.05720
 version number is 1
5063
https://export.arxiv.org/abs/2001.05721
 version number is 2
http

In [ ]:
Looking into some content. 

In [80]:
print(df3.loc[:,['Id','Abstract-number','Final','version_number']])

              Id Abstract-number  Final version_number
0     2001.00658      2001.00658   True              1
1     2001.00659      2001.00659   True              2
2   2001.00659v1      2001.00659  False              1
3     2001.00660      2001.00660   True              1
4     2001.00661      2001.00661   True              1
5     2001.00662      2001.00662   True              2
6   2001.00662v1      2001.00662  False              1
7     2001.00663      2001.00663   True              1
8     2001.00664      2001.00664   True              5
9   2001.00664v4      2001.00664  False              4
10  2001.00664v3      2001.00664  False              3
11  2001.00664v2      2001.00664  False              2
12  2001.00664v1      2001.00664  False              1
13    2001.00665      2001.00665   True              2
14  2001.00665v1      2001.00665  False              1
15    2001.00666      2001.00666   True              1
16    2001.00667      2001.00667   True              1
